# "Model Based Reinforcement Learning (MBRL)"
> ""

- toc:true
- branch: master
- badges: true
- comments: true
- author: Isaac Kargar
- categories: [jupyter]
- show_image: true


## Introduction and Motivation

Having access to a model of the world and using it for decision making is a powerful idea. 
There are a lot of applications of MBRL in different areas like robotics (manipulation- what will happen by doing an action), 
self-driving cars (having a model of other agents decisions and future motions and act accordingly),
games (AlphaGo- search over different possibilities), Science ( chemical usecases),
and peration research and energy applications (how to allocate renewable energy in different points in time to meet the demand).

## Problem Statement

In sequential decision making, the agent will interact with the world by doing action $a$ and getting the next state $s$ and reward $r$.


We can write this problem as a Markov Decision Process (MDP) as follows:

- States $S \epsilon R^{d_S}$
- Actions $A \epsilon R^{d_A}$
- Reward function $R: S \times A \rightarrow R$
- Transition function $T: S \times A \rightarrow S$
- Discount $\gamma \epsilon (0,1)$
- Policy $\pi: S \rightarrow A$